In [156]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [157]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [158]:
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
submission_sample = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
cats = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

# Cats 

In [159]:
#카테고리 코드정리
#1. 문자열 분리
cats['split'] = cats['item_category_name'].str.split('-')
#2. [0]인 값만 사용, [1]의 unique()가 너무 많고, 최대 중복이 4여서 버림
cats['type'] = cats['split'].map(lambda x: x[0].strip())

#3. 카테고리 병합(PC와 모바일 게임도 게임으로 재분류)
cats.loc[cats['type']=='Игры PC','type'] = 'Игры'
cats.loc[cats['type']=='Игры MAC','type'] = 'Игры'
cats.loc[cats['type']=='Игры Android''type'] = 'Игры'

#4. 중복이 4보다 적은 값은 orters로 분류 -> label or onehot을 위해 최소화
cat_types =[]
for i in cats['type'].unique():
    if len(cats[cats['type'] == i]) >= 4:
        cat_types.append(i)
cats['type'] = cats['type'].apply(lambda x: x if (x in cat_types) else "orters")


In [160]:
#5. 라벨인코더 
from sklearn.preprocessing import LabelEncoder
hot_model = LabelEncoder()
cats['type_1'] = hot_model.fit_transform(cats['type'])

#6. 병합할 데이터 셋
#병합시 Key(on) = item_category_id
cats_final = pd.DataFrame({'item_category_id':cats['item_category_id'],'type':cats['type_1']})

# item

In [161]:
#item name은 cats의 type과 성질이 유사하여 삭제
items = pd.merge(items,cats_final, on='item_category_id', how="left")
items = items.drop('item_name', axis=1)

# shop

In [162]:
# train/test의 shop_id열에서 shop_name이 중복인 값의 shop_id 통일
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11
# shops의 shop_id열에서도 shop_name이 중복인 값의 shop_id 통일
shops.loc[shops.shop_id == 0, 'shop_id'] = 57
shops.loc[shops.shop_id == 0, 'shop_id'] = 57
shops.loc[shops.shop_id == 1, 'shop_id'] = 58
shops.loc[shops.shop_id == 1, 'shop_id'] = 58
shops.loc[shops.shop_id == 10, 'shop_id'] = 11
shops.loc[shops.shop_id == 10, 'shop_id'] = 11
# shop_name에서 분류가 안되는 값들 분류가 잘 되도록 수정하고, (city나 category명 없거나 띄어쓰기 이상하게 되있거나 한 값들이 꽤 있어서)
shops.iloc[0,0] = 'Якутск Орджоникидзе, 56 фран	'
shops.iloc[1,0] = 'Якутск ТЦ "Центральный" фран'
shops.iloc[6,0] = 'Воронеж  (Плехановская, 13)'
shops.iloc[9,0] = ' Выездная Торговля '
shops.iloc[10,0] = 'Жуковский  ул.Чкалова39м'
shops.iloc[11,0] = 'Жуковский  ул.Чкалова39м²'
shops.iloc[12,0] = ' Интернет-магазин ЧС'
shops.iloc[20,0] = 'Москва  "Распродажа"'
shops.iloc[46,0] = 'СергиевПосад ТЦ "7Я"'
shops.iloc[55,0] = ' Цифровойсклад1С-Онлайн '
shops.iloc[57,0] = 'Якутск  Орджоникидзе'
shops['category'] = shops.shop_name.str.split(' ').map(lambda x: x[1])
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
# shop_id에서 id가 중복되는 행 삭제 (미정)
shops.drop_duplicates(['shop_id'],inplace = True)
# categoty 항목이 5개 미만이거나 공백인 값들은 other로 변경
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5 and cat != '':
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )
# ["shop_id", "shop_category", "shop_city"] 행만 남게 한 후 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
shops['shop_category'] = LabelEncoder().fit_transform(shops.category)
shops['shop_city'] = LabelEncoder().fit_transform(shops.city)
shops = shops[["shop_id", "shop_category", "shop_city"]]

In [163]:
shops

,shop_id,shop_category,shop_city
0,57,0,27
1,58,4,27
2,2,4,1
3,3,2,2
4,4,4,3
5,5,3,4
6,6,0,5
7,7,3,5
8,8,3,5
9,9,0,0


# train

In [164]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [165]:
#정리
train = train[(train['item_price'] < 100000) & (train['item_price'] > 0)& (train['item_cnt_day'] <1000)]

In [166]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935845 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 156.8+ MB


In [167]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

In [168]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


# 데이터 병합

In [169]:
#lag 만들기
def lag_data( df,lags, cols ):
    for col in cols:
        print(col)
        tmp = df[["date_block_num", "shop_id","item_id",col ]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df
def select_trends(row) :
    for i in lags:
        if row["delta_price_lag_" + str(i)]:
            return row["delta_price_lag_" + str(i)]
    return 0

In [170]:
#product = 최대순열 만드는 함수
from itertools import product
matrix = []
cols = ['date_block_num','shop_id','item_id']
matrix = []
for i in range(34):
    sales = train[train.date_block_num == i]
    matrix.append( np.array(list( product( [i], sales.shop_id.unique(), sales.item_id.unique() ) ), dtype = np.int16) )

matrix = pd.DataFrame( np.vstack(matrix), columns = ["date_block_num", "shop_id", "item_id"] )
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values( ["date_block_num", "shop_id", "item_id"], inplace = True )

In [171]:
group = train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
group.columns = ["item_cnt_month"]
group.reset_index( inplace = True)
matrix = pd.merge( matrix, group, on = ["date_block_num", "shop_id", "item_id"], how = "left" )
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).astype(np.float16)

In [172]:
test["date_block_num"] = 34
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test["shop_id"] = test.shop_id.astype(np.int8)
test["item_id"] = test.item_id.astype(np.int16)

In [173]:
matrix = pd.concat([matrix, test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=["date_block_num", "shop_id", "item_id"])
matrix.fillna( 0, inplace = True )

In [174]:
matrix = pd.merge( matrix, shops, on = ["shop_id"], how = "left" )
matrix = pd.merge(matrix, items, on = ["item_id"], how = "left")
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128004 entries, 0 to 11128003
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   date_block_num    int8   
 1   shop_id           int8   
 2   item_id           int16  
 3   item_cnt_month    float16
 4   shop_category     int64  
 5   shop_city         int64  
 6   item_category_id  object 
 7   type              int64  
dtypes: float16(1), int16(1), int64(3), int8(2), object(1)
memory usage: 488.2+ MB


In [175]:
matrix.isnull().sum()

date_block_num      0
shop_id             0
item_id             0
item_cnt_month      0
shop_category       0
shop_city           0
item_category_id    0
type                0
dtype: int64

In [176]:
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["type"] = matrix["type"].astype(np.int8)

In [177]:
group = train.groupby( ["item_id"] ).agg({"item_price": ["mean"]})
group.columns = ["item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge( group, on = ["item_id"], how = "left" )
matrix["item_avg_item_price"] = matrix.item_avg_item_price.astype(np.float16)

In [178]:
group = train.groupby( ["date_block_num","item_id"] ).agg( {"item_price": ["mean"]} )
group.columns = ["date_item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge(group, on = ["date_block_num","item_id"], how = "left")
matrix["date_item_avg_item_price"] = matrix.date_item_avg_item_price.astype(np.float16)
lags = [1, 2, 3]
matrix = lag_data( matrix, lags, ["date_item_avg_item_price"] )
for i in lags:
    matrix["delta_price_lag_" + str(i) ] = (matrix["date_item_avg_item_price_lag_" + str(i)]- matrix["item_avg_item_price"] )/ matrix["item_avg_item_price"]


date_item_avg_item_price


In [179]:
#아이템가격이 1개인 경우도 있고, 여러개인 경우도 있다.
item_id_list = matrix['item_id'].unique()
lis_price = {}
for i in item_id_list:
    pwd = matrix[matrix['item_id']==i]['date_item_avg_item_price']
    lis_price[i]=len(pwd.value_counts())

In [180]:
item_id_list = matrix['item_id'].unique()
item_id_list.sort()
item_id_list
sample = []
for i in item_id_list:
    sample.append(lis_price[i])
price_merge_sample = pd.DataFrame({'item_id':np.array(item_id_list),'price_unique':np.array(sample)})

In [181]:
price_merge_sample[price_merge_sample['price_unique']>5]

,item_id,price_unique
27,27,9
28,28,8
30,30,18
31,31,24
32,32,21
...,...,...
22154,22155,7
22159,22160,6
22161,22162,7
22163,22164,7


In [182]:
matrix = pd.merge(matrix, price_merge_sample, on = ["item_id"], how = "left" )
matrix["price_unique"] = matrix["price_unique"].fillna(0).astype(np.int16)

In [183]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128004 entries, 0 to 11128003
Data columns (total 17 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   date_block_num                  int8   
 1   shop_id                         int8   
 2   item_id                         int16  
 3   item_cnt_month                  float16
 4   shop_category                   int8   
 5   shop_city                       int8   
 6   item_category_id                object 
 7   type                            int8   
 8   item_avg_item_price             float16
 9   date_item_avg_item_price        float16
 10  date_item_avg_item_price_lag_1  float16
 11  date_item_avg_item_price_lag_2  float16
 12  date_item_avg_item_price_lag_3  float16
 13  delta_price_lag_1               float16
 14  delta_price_lag_2               float16
 15  delta_price_lag_3               float16
 16  price_unique                    int16  
dtypes: float16(9), int16(2), 

In [184]:
matrix['Dec_dummy'] = matrix['date_block_num'].apply(lambda x: 1 if x%12==0 else 0)

In [185]:
matrix["item_category_id"] = matrix["item_category_id"].fillna(0).astype(np.int16)

In [186]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month,shop_category,shop_city,item_category_id,type,item_avg_item_price,date_item_avg_item_price,date_item_avg_item_price_lag_1,date_item_avg_item_price_lag_2,date_item_avg_item_price_lag_3,delta_price_lag_1,delta_price_lag_2,delta_price_lag_3,price_unique,Dec_dummy
0,0,2,19,0.0,4,1,40,5,28.000,28.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,0,2,27,1.0,4,1,19,3,1461.000,2324.0,NaN,NaN,NaN,NaN,NaN,NaN,9,1
2,0,2,28,0.0,4,1,30,3,310.000,549.0,NaN,NaN,NaN,NaN,NaN,NaN,8,1
3,0,2,29,0.0,4,1,23,3,1759.000,2398.0,NaN,NaN,NaN,NaN,NaN,NaN,5,1
4,0,2,32,0.0,4,1,40,5,249.625,338.0,NaN,NaN,NaN,NaN,NaN,NaN,21,1


In [187]:
#피쳐 늘리기
matrix["delta_price_lag"] = matrix.apply(select_trends, axis = 1)
matrix["delta_price_lag"] = matrix.delta_price_lag.astype( np.float16 )
matrix["delta_price_lag"].fillna( 0 ,inplace = True)


features_to_drop = ["item_avg_item_price", "date_item_avg_item_price"]
for i in lags:
    features_to_drop.append("date_item_avg_item_price_lag_" + str(i) )
    features_to_drop.append("delta_price_lag_" + str(i) )
matrix.drop(features_to_drop, axis = 1, inplace = True)

In [188]:
group = train.groupby( ["shop_id"] ).agg({"revenue": ["sum"] })
group.columns = ["shop_revenue"]
group.reset_index(inplace = True)

matrix = matrix.merge( group , on = ["shop_id"], how = "left" )
matrix['shop_revenue'] = matrix['shop_revenue'].astype(np.float32)


group = train.groupby( ["date_block_num","shop_id"] ).agg({"revenue": ["sum"] })
group.columns = ["date_shop_revenue"]
group.reset_index(inplace = True)

matrix = matrix.merge( group , on = ["date_block_num", "shop_id"], how = "left" )
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(["shop_id"]).agg({ "date_block_num":["mean"] })
group.columns = ["shop_avg_revenue"]
group.reset_index(inplace = True )

matrix = matrix.merge( group, on = ["shop_id"], how = "left" )
matrix["shop_avg_revenue"] = matrix.shop_avg_revenue.astype(np.float32)
matrix["delta_revenue"] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix["delta_revenue"] = matrix["delta_revenue"]. astype(np.float32)

matrix = lag_data(matrix, [1], ["delta_revenue"])
matrix["delta_revenue_lag_1"] = matrix["delta_revenue_lag_1"].astype(np.float32)
matrix.drop( ["date_shop_revenue", "shop_avg_revenue", "delta_revenue"] ,axis = 1, inplace = True)

delta_revenue


In [189]:
matrix = lag_data(matrix, [1,2,3], ['item_cnt_month'])

item_cnt_month


In [190]:

# group = train.groupby( ["date_block_num","shop_id"] ).agg({"revenue": ["sum"] })
# group.columns = ["date_shop_revenue"]
# group.reset_index(inplace = True)

# matrix = matrix.merge( group , on = ["date_block_num", "shop_id"], how = "left" )
# matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

In [191]:
#가격 변동이 있음으로 -> 매출로 변경해야 겠다 <- 위에서 수행했음으로 주석처리
#확인명령어
# train.groupby(["item_id"])['item_price'].sum()
# train[train['item_id']==22167]['item_price'].value_counts()
# train[(train['item_id']==22167) & (train['item_price']==155.00)]

# #가격 극단치 확인, 6066이라는 id를 삭제
# train[train['item_price']>=100000]
# train[(train['item_id']==6066)]

In [194]:
# matrix = lag_data(matrix, [1,2,3], ["revenue"])

In [197]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month,shop_category,shop_city,item_category_id,type,price_unique,Dec_dummy,delta_price_lag,shop_revenue,delta_revenue_lag_1,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,2,19,0.0,4,1,40,5,1,1,0.000000,44049640.0,NaN,NaN,NaN,NaN
1,0,2,27,1.0,4,1,19,3,9,1,0.000000,44049640.0,NaN,NaN,NaN,NaN
2,0,2,28,0.0,4,1,30,3,8,1,0.000000,44049640.0,NaN,NaN,NaN,NaN
3,0,2,29,0.0,4,1,23,3,5,1,0.000000,44049640.0,NaN,NaN,NaN,NaN
4,0,2,32,0.0,4,1,40,5,21,1,0.000000,44049640.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127999,34,45,18454,0.0,4,19,55,7,7,0,-0.475098,40924796.0,51790.574219,1.0,0.0,0.0
11128000,34,45,16188,0.0,4,19,64,8,2,0,0.081116,40924796.0,51790.574219,0.0,0.0,NaN
11128001,34,45,15757,0.0,4,19,55,7,9,0,0.155884,40924796.0,51790.574219,0.0,0.0,0.0
11128002,34,45,19648,0.0,4,19,40,5,3,0,-0.091736,40924796.0,51790.574219,0.0,0.0,0.0


In [198]:
# matrix = lag_data(matrix, [1,2,3], ["item_price"])

In [199]:
matrix = lag_data(matrix, [1], ["Dec_dummy"])

Dec_dummy


In [200]:
matrix = matrix[matrix["date_block_num"] > 3]
data = matrix.copy()

In [201]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9630539 entries, 1497465 to 11128003
Data columns (total 17 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date_block_num        int8   
 1   shop_id               int8   
 2   item_id               int16  
 3   item_cnt_month        float16
 4   shop_category         int8   
 5   shop_city             int8   
 6   item_category_id      int16  
 7   type                  int8   
 8   price_unique          int16  
 9   Dec_dummy             int64  
 10  delta_price_lag       float16
 11  shop_revenue          float32
 12  delta_revenue_lag_1   float32
 13  item_cnt_month_lag_1  float16
 14  item_cnt_month_lag_2  float16
 15  item_cnt_month_lag_3  float16
 16  Dec_dummy_lag_1       float64
dtypes: float16(5), float32(2), float64(1), int16(3), int64(1), int8(5)
memory usage: 486.8 MB


In [202]:
data.corr()

,date_block_num,shop_id,item_id,item_cnt_month,shop_category,shop_city,item_category_id,type,price_unique,Dec_dummy,delta_price_lag,shop_revenue,delta_revenue_lag_1,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,Dec_dummy_lag_1
date_block_num,1.000000,0.022619,-0.012621,-0.006062,-0.032212,0.021061,0.016756,0.006513,0.037599,0.004344,-0.146137,-0.031271,-0.013851,0.000136,0.000026,0.001251,0.036210
shop_id,0.022619,1.000000,-0.000345,0.008253,0.100518,0.872778,-0.000196,-0.000440,0.002746,0.002864,-0.005231,0.035610,0.050879,0.010093,0.010810,0.011386,0.001955
item_id,-0.012621,-0.000345,1.000000,-0.012190,0.000333,-0.000299,0.129375,0.148790,-0.019263,-0.002648,-0.007435,0.000286,-0.001810,-0.015998,-0.017459,-0.018325,-0.001088
item_cnt_month,-0.006062,0.008253,-0.012190,1.000000,-0.001670,0.007051,-0.025885,-0.036748,0.082321,0.001676,0.006678,0.065251,0.061038,0.726736,0.649696,0.626879,0.000640
shop_category,-0.032212,0.100518,0.000333,-0.001670,1.000000,0.217179,-0.000936,-0.000621,-0.000342,0.003402,0.003727,-0.059247,-0.027771,0.000031,0.001068,0.001632,0.003451
shop_city,0.021061,0.872778,-0.000299,0.007051,0.217179,1.000000,-0.000259,-0.000481,0.002890,0.003506,-0.004470,0.033767,0.053147,0.010399,0.011784,0.012496,0.001359
item_category_id,0.016756,-0.000196,0.129375,-0.025885,-0.000936,-0.000259,1.000000,0.955193,-0.246904,-0.005680,0.047133,-0.000230,0.000829,-0.029875,-0.031910,-0.033169,-0.000694
type,0.006513,-0.000440,0.148790,-0.036748,-0.000621,-0.000481,0.955193,1.000000,-0.236091,-0.006944,0.048657,0.000046,0.000346,-0.042806,-0.045710,-0.047734,-0.001482
price_unique,0.037599,0.002746,-0.019263,0.082321,-0.000342,0.002890,-0.246904,-0.236091,1.000000,0.012170,-0.049573,-0.002853,0.006114,0.084160,0.089743,0.092755,0.019821
Dec_dummy,0.004344,0.002864,-0.002648,0.001676,0.003402,0.003506,-0.005680,-0.006944,0.012170,1.000000,-0.008014,0.000158,0.365603,0.015970,0.003495,-0.000839,-0.073455


In [203]:
data.isnull().sum()

date_block_num                0
shop_id                       0
item_id                       0
item_cnt_month                0
shop_category                 0
shop_city                     0
item_category_id              0
type                          0
price_unique                  0
Dec_dummy                     0
delta_price_lag               0
shop_revenue                  0
delta_revenue_lag_1     1957843
item_cnt_month_lag_1    1957843
item_cnt_month_lag_2    2252408
item_cnt_month_lag_3    2539771
Dec_dummy_lag_1         1957843
dtype: int64

In [204]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

# Modeling

In [205]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor()

In [206]:
# params = {
#     'task':['predict'],
#     'boosting_type':['gbdt','dark'],
#     'objective': ['root_mean_squared_error'],
#     'max_depth' :[ 10, 15, 25],
#     'learning_rate':[0.2, 0.1, 0.05],
#     'n_estimators':[1000,2000]}
# #https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

# gv_model = GridSearchCV(estimator=lgb_model, param_grid=params, cv=5)
# gv_model.fit(X_train, Y_train)

# gv_model.best_params_

In [207]:
model = lgb.LGBMRegressor(#task='predict',
    boostin_type = 'gbdt',#dark
    #objective= 'root_mean_squared_error',
    num_iterations =1000, #반복량
    #min_child_weight=0.5, #자식노드 최소w
    # max_depth = 1000,
    learning_rate=0.1, #defualt=0.1
    n_estimators = 1000
)


In [209]:
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 20)
#https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.early_stopping.html

[1]	training's rmse: 1.16929	training's l2: 1.36725	valid_1's rmse: 1.10453	valid_1's l2: 1.21999
[2]	training's rmse: 1.13024	training's l2: 1.27744	valid_1's rmse: 1.07818	valid_1's l2: 1.16247
[3]	training's rmse: 1.09701	training's l2: 1.20343	valid_1's rmse: 1.05678	valid_1's l2: 1.11679
[4]	training's rmse: 1.06916	training's l2: 1.14311	valid_1's rmse: 1.03922	valid_1's l2: 1.07997
[5]	training's rmse: 1.04528	training's l2: 1.09261	valid_1's rmse: 1.0255	valid_1's l2: 1.05165
[6]	training's rmse: 1.0248	training's l2: 1.0502	valid_1's rmse: 1.01402	valid_1's l2: 1.02823
[7]	training's rmse: 1.00778	training's l2: 1.01563	valid_1's rmse: 1.0054	valid_1's l2: 1.01082
[8]	training's rmse: 0.992926	training's l2: 0.985903	valid_1's rmse: 0.997022	valid_1's l2: 0.994052
[9]	training's rmse: 0.980341	training's l2: 0.961069	valid_1's rmse: 0.991175	valid_1's l2: 0.982428
[10]	training's rmse: 0.970148	training's l2: 0.941187	valid_1's rmse: 0.986236	valid_1's l2: 0.972661
[11]	traini

LGBMRegressor(boostin_type='gbdt', n_estimators=1000, num_iterations=1000)

In [210]:
y_pred = model.predict(X_valid)
y_test = model.predict(X_test)

In [211]:
y_pred.mean()

0.24985049878981214

In [212]:
y_test.mean()

0.2591789069500448

In [214]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": y_test
})
submission.to_csv('lightGBM_submission.csv', index=False)

In [ ]:
#data_1: item  정보  ['itemaggregate_name', 'item_id', 'item_category_id'], dtype='object'
#data_2: item 월 판매량 ['ID', 'item_cnt_month'], dtype='object'
#data_3: item 카테고리 정보['item_category_name', 'item_category_id'], dtype='object'
#data_4: 학습용 데이터(일판매량 ): ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price','item_cnt_day'
#data_5: shop  정보 ['shop_name', 'shop_id']
#data_6: test셋 ['ID', 'shop_id', 'item_id'], dtype='object'_____